# CBIP Study

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm, trange
from pyvis.network import Network
from coloring.CBIP import cbip
from utils import generate_k_colorable_graph

### Specify parameters here

In [2]:
N = 10    # Number of online k-colorable graphs to generate at each step
n_range = range(100,1601,100)

In [3]:
def convert_to_dataframe(data:list) -> pd.DataFrame:
    df = pd.DataFrame(data, columns=['k', 'n', 'colors_used', 'ratio', 'N'])
    return df

In [4]:
# <k, n, colors_used, ratio, N>
data = []
for n in tqdm(n_range):
    ratios = []
    for _ in range(N):
        # Initialize the graph
        G = generate_k_colorable_graph(2, n, 0.9)

        # Simulate online coloring
        for idx in range(2, n+1):
            subgraph = G.subgraph(range(idx))
            try:
                colors = cbip(subgraph)
            except:
                print('Skipping')
                continue
            # Color the graph
            for node, color in colors.items():
                G.nodes[node]['group'] = color
            
        # Determine the number of colors used
        colors_used = len(set(colors.values()))

        # Determine the ratio of colors used
        ratio = colors_used / 2
        ratios.append(ratio)

    # Append the data
    data.append([2, n, colors_used, np.mean(ratios), N])
    # Save checkpoint
    _df = convert_to_dataframe(data)
    _df.to_feather(f'../data/cbip_checkpoint_2.ftr')

# Save the data
df = convert_to_dataframe(data)
df.to_feather(f'../data/cbip_2.ftr')

 50%|█████     | 8/16 [16:43<30:14, 226.85s/it]